In [1]:
import re

In [ ]:
with open ('./text.txt','r',encoding='utf-8') as file:
    text = file.read()

In [3]:
all_words = re.split(r'([,.;?_!"()\']|--|\s)',text)
len(all_words)

9195

In [4]:
all_words.extend(['<|EOT|>'])
all_words = list(set(all_words))
len(all_words)

1150

In [5]:
def bpe_breaker(st):
    subsets = []
    for i in range(len(st)):
        for j in range(len(st)-i+1):
            if st[i:i+j]:
                subsets.append(st[i:i+j])
    subsets = sorted(set(subsets), key=len, reverse=True)
    return subsets


In [6]:
all_word_combos = []

In [7]:
for word in all_words:
    all_word_combos.extend(bpe_breaker(word))

In [8]:
all_word_combos = set(sorted(all_word_combos))
len(all_word_combos)

11688

In [9]:
vocab = {token:index for index,token in enumerate(all_word_combos) if token}

In [21]:
class bpe_tokenizer:
    def __init__(self):
        self.s2i = vocab
        self.i2s = {index:token for token,index in vocab.items()}
    def bpe_breaker(self, st=''):
        subsets = []
        for i in range(len(st)):
            for j in range(len(st),0,-1):
                if st[i:j]:
                    subsets.append(st[i:j])
        return subsets
    def encode(self, input_text):
        ids = []
        input_text+="<|EOT|>"
        inp = re.split(r'([,.;?_!"()\']|--|\s)',input_text)
        for word in inp:
            while word != "":
                # print(f"--->{word}")
                try:
                    if self.s2i[word]:
                        ids.append(self.s2i[word])
                        break
                except:
                    for part in self.bpe_breaker(word):
                        try:
                            if self.s2i[part]:
                                ids.append(self.s2i[part])
                                word = word[:word.index(part)]+word[word.index(part)+len(part):]
                                break
                        except Exception as e:
                            print(f" -- ERROR --> {e} ")
                            pass
        return ids
    def decode(self, ids):
        for i in ids:
            text = "".join([self.i2s[i] for i in ids])
            text = re.sub(r'\s+([,.?!"()\'])', r'\1', text).replace("' ","'")
        return text

In [22]:
string_ = "His asbsaggav confident eyes grew dim, and his cheeks paled a little under their h."
bpe = bpe_tokenizer()
encoded_tokens = bpe.encode(string_)
decoded_ids = bpe.decode(encoded_tokens)

print(len(encoded_tokens))
print(string_)
print(decoded_ids)



 -- ERROR --> 'asbsaggav' 
 -- ERROR --> 'asbsagga' 
 -- ERROR --> 'asbsagg' 
 -- ERROR --> 'asbsag' 
 -- ERROR --> 'asbsa' 
 -- ERROR --> 'asbs' 
 -- ERROR --> 'asb' 
 -- ERROR --> 'bsaggav' 
 -- ERROR --> 'bsagga' 
 -- ERROR --> 'bsagg' 
 -- ERROR --> 'bsag' 
 -- ERROR --> 'bsa' 
 -- ERROR --> 'aggav' 
 -- ERROR --> 'agga' 
35
His asbsaggav confident eyes grew dim, and his cheeks paled a little under their h.
His asbsaggav confident eyes grew dim, and his cheeks paled a little under their h.<|EOT|>
